In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
%pip install torchinfo
%restart_python

In [0]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from bacp import BaCPLearner, BaCPTrainer, BaCPTrainingArgumentsLLM
from models import EncoderProjectionNetwork, ClassificationNetwork
from unstructured_pruning import MagnitudePrune, MovementPrune, LocalMagnitudePrune, LocalMovementPrune, WandaPrune, PRUNER_DICT, check_model_sparsity
from LLM_trainer import LLMTrainer, LLMTrainingArguments
from dataset_utils import get_glue_data
from logger import Logger

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments

from tqdm import tqdm
from torchinfo import summary

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 

from utils import *
from constants import *

device = get_device()
print(f"{device = }")
BATCH_SIZE_DISTILBERT = 64
NUM_WORKERS = 24


# Baseline Accuracies

## SST-2 Dataset

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

training_args = LLMTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    epochs=3,
    learning_type="baseline",
    prune=False,
)

trainer = LLMTrainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")



# Pruning Accuracies

## SST-2 Dataset

### Magnitude Prune

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_LOW

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)

trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_MID

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)

trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_HIGH

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)

trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

### Movement Prune

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_LOW

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)
trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_MID

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)

trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_HIGH

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)
trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

### WandA Prune

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "wanda_pruning"
target_sparsity = TARGET_SPARSITY_LOW

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)
trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "wanda_pruning"
target_sparsity = TARGET_SPARSITY_MID

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)

trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

In [0]:
# Model initialization
model_name = "distilbert-base-uncased"
model_task = "sst2"

# Loading trained weights
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

# Initializing pruning method
pruning_type = "wanda_pruning"
target_sparsity = TARGET_SPARSITY_HIGH

training_args = LLMTrainingArguments(
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    pruning_scheduler="cubic",
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    learning_type='pruning',
    learning_rate=1e-5,
    epochs=5,
    recovery_epochs=10,
)

trainer = LLMTrainer(training_args=training_args)

if True:
    trainer.train()

acc = trainer.evaluate()
print(f"Accuracy = {acc}")

# BaCP Accuracies

## SST-2

### Magnitude Pruning

In [0]:
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_LOW
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")



In [0]:
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_MID
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")



In [0]:
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_HIGH
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")



### Movement Pruning

In [0]:
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_LOW
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")



In [0]:
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_MID
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")



In [0]:
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_HIGH
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")


### Wanda Pruning

In [0]:
pruning_type = "wanda_pruning"
target_sparsity = TARGET_SPARSITY_LOW
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")



In [0]:
pruning_type = "wanda_pruning"
target_sparsity = TARGET_SPARSITY_MID
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")



In [0]:
pruning_type = "wanda_pruning"
target_sparsity = TARGET_SPARSITY_HIGH
model_name = "distilbert-base-uncased"
model_task = "sst2"
trained_model_path = f"/dbfs/research/{model_name}/{model_task}/{model_name}_baseline.pt"

bacp_training_args = BaCPTrainingArgumentsLLM(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_DISTILBERT,
    finetuned_weights=trained_model_path,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_rate=1e-5,
    )
bacp_trainer = BaCPTrainer(bacp_training_args)
if True:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
pruner = bacp_trainer.get_pruner()

llm_training_args = LLMTrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    finetuned_weights=bacp_trainer.cm_save_path,
    epochs=3,
    prune=False,
    pruner=pruner,
    finetune=True,
    learning_type="bacp_finetune"
)
llm_trainer = LLMTrainer(llm_training_args)
if True:
    llm_trainer.train()

acc = llm_trainer.evaluate()
print(f"Accuracy = {acc}")

